## 07_2. Myeloid -- Milo - KNN based differential abundance analysis

<div 
    <p style="text-align: left;">Updated Time: 2025-02-20</p>
</div>

##### Load libraries

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import scanpy as sc
import pertpy as pt

import matplotlib as mpl
from matplotlib import font_manager as fm
import matplotlib.pyplot as plt

fm.fontManager.addfont("/usr/share/fonts/truetype/msttcorefonts/Arial.ttf")
mpl.rcParams.update({
    "font.family": ["Arial", "Noto Sans", "DejaVu Sans"],
    "mathtext.fontset": "dejavusans",
    "axes.unicode_minus": False,
    "pdf.fonttype": 42,
    "svg.fonttype": "none",
})

import warnings
warnings.simplefilter("ignore")

##### Set working directory for analysis

In [ ]:
cwd = '/media/bio/Disk/Research Data/EBV/omicverse'
os.chdir(cwd)
updated_dir = os.getcwd()
print("Updated working directory: ", updated_dir)

##### Reading in annotated AnnData object

In [ ]:
adata = sc.read_h5ad("Processed Data/scRNA_TCell.h5ad")
adata

In [ ]:
for i in adata.obs['T_subtype'].cat.categories:
  number = len(adata.obs[adata.obs['T_subtype']==i])
  print('the number of category {} is {}'.format(i,number))

In [ ]:
for i in adata.obs['EBV_status'].cat.categories:
  number = len(adata.obs[adata.obs['EBV_status']==i])
  print('the number of category {} is {}'.format(i,number))

### Milo - KNN based differential abundance analysis

##### Prepare for Milo analysis

In [ ]:
## Exclude Normal samples
adata = adata[adata.obs["EBV_status"] != "Normal"].copy()

## Initialize object for Milo analysis
milo = pt.tl.Milo()
mdata = milo.load(adata)

When initializing the Milo object, we create a MuData object which will store both the gene expression matrices (rna view) and cell count matrices used for differential abundance analysis (milo view).

In [ ]:
mdata

#### Build KNN graph
We can use scanpy functions to build a KNN graph. We set the dimensionality and value for k to use in subsequent steps.

Here the value of k indicates the smallest possible size of neighbourhood in which we will quantify differential abundance (i.e. with k=50 the smallest neighbourhood will have 50 cells). Depending on the number of samples, you might want to use a high value of k for neighbourhood analysis, to have sufficient power to estimate abundance fold-changes. Since here we have data from > 100 patients, we set k=150 to have on average more than one cell per donor in each neighbourhood.

In [ ]:
sc.pp.neighbors(mdata["rna"], use_rep="X_harmony", n_neighbors=15)

#### Construct neighbourhoods


This step assigns cells to a set of representative neighbourhoods on the KNN graph.



In [ ]:
milo.make_nhoods(mdata["rna"], prop=0.1)

The assignment of cells to neighbourhoods is stored as a sparse binary matrix in mdata['rna'].obsm. Here we see that cells have been assigned to 4307 neighbourhoods.


In [ ]:
mdata["rna"].obsm["nhoods"]

The information on which cells are sampled as index cells of representative neighbourhoods is stored in mdata['rna'].obs, along with the distance of the index to the kth nearest neighbor, which is used later for the SpatialFDR correction.

In [ ]:
mdata["rna"][mdata["rna"].obs["nhood_ixs_refined"] != 0].obs[["nhood_ixs_refined", "nhood_kth_distance"]]

We can visualize the distribution of neighbourhood sizes, to check that the minimal value of k makes sense, and that the distribution of sizes is not too wide.

In [ ]:
nhood_size = np.array(mdata["rna"].obsm["nhoods"].sum(0)).ravel()
plt.hist(nhood_size, bins=100)
plt.xlabel("# cells in nhood")
plt.ylabel("# nhoods");

##### Count cells in neighbourhoods
Milo leverages the variation in cell numbers between replicates for the same experimental condition to test for differential abundance. Therefore we have to count how many cells from each sample (in this case the patient) are in each neighbourhood. We need to use the cell metadata saved in adata.obs and specify which column contains the sample information.

In [ ]:
mdata = milo.count_nhoods(mdata, sample_col="orig.ident")

This function populates the modality milo to mdata.

mdata[‘milo’] is an anndata object where obs correspond to samples and vars correspond to neighbourhoods, and where .X stores the number of cells from each sample counted in a neighbourhood. This count matrix will be used for DA testing.

In [ ]:
mdata

In [ ]:
mdata["milo"]

##### Differential abundance testing with GLM
We are now ready to test for differential abundance in time. The experimental design needs to be specified with R-style formulas.

Here we run a simple comparison, testing for changes in cell abundance associated with EBV+ NPC.

In [ ]:
# Reorder categories
# (by default, the last category is taken as the condition of interest)
mdata["rna"].obs["EBV_status"] = mdata["rna"].obs["EBV_status"].cat.reorder_categories(["Negative", "Positive"])
milo.da_nhoods(mdata, design="~EBV_status")

We can explicitly specify the comparison we want to make using the parameter model_contrasts. This is especially important when we have more than two levels for a category and we want to compare against a specific control level.

In [ ]:
milo.da_nhoods(mdata, design="~EBV_status", model_contrasts="EBV_statusPositive-EBV_statusNegative")

In addition, we might want to take into account potential confounders that could affect cell abundances, for example batch effects. We can include confounders in the model using the syntax ~ confounders + condition, where the covariate specified in the last term is always the covariate of interest. In this case we can estimate the effect of EBV infection on cell abundance regressing out changes in cell abundance driven by the site of collection.

See below for more examples on how to specify different types of comparisons.

In [ ]:
milo.da_nhoods(mdata, design="~Dataset+EBV_status", model_contrasts="EBV_statusPositive")

Information about the sample design is stored in mdata['milo'].obs:

In [ ]:
mdata["milo"].obs

The differential abundance test results are stored in milo_mdata['milo'].var. In particular:

logFC: stores the log-Fold Change in abundance (i.e. the slope of the linear model)

PValue stores the p-value for the test

SpatialFDR stores the p-values adjusted for multiple testing (accounting for overlap between neighbourhoods)

In [ ]:
mdata["milo"].var

We can start inspecting the results of our DA analysis from a couple of standard diagnostic plots.

In [ ]:
old_figsize = plt.rcParams["figure.figsize"]
plt.rcParams["figure.figsize"] = [10, 5]
plt.subplot(1, 2, 1)
plt.hist(mdata["milo"].var.PValue, bins=50)
plt.xlabel("P-Vals")
plt.subplot(1, 2, 2)
plt.plot(mdata["milo"].var.logFC, -np.log10(mdata["milo"].var.SpatialFDR), ".")
plt.xlabel("log-Fold Change")
plt.ylabel("- log10(Spatial FDR)")
plt.tight_layout()
plt.rcParams["figure.figsize"] = old_figsize

#### Visualize results on embedding
To visualize DA results relating them to the embedding of single cells, we can build an abstracted graph of neighbourhoods that we can superimpose on the single-cell embedding. Here each node represents a neighbourhood, and the layout of nodes is determined by the position of the index cell in the UMAP embedding of all single-cells. The neighbourhoods displaying singificant DA are colored by their log-Fold Change.

In [ ]:
milo.build_nhood_graph(mdata)

In [ ]:
mdata

In [ ]:
plt.rcParams["figure.figsize"] = [7, 7]
milo.plot_nhood_graph(
    mdata,
    alpha=0.1,  ## SpatialFDR level (1%)
    min_size=1,  ## Size of smallest dot
)

#### Visualize result by celltype
We might want to visualize whether DA is particularly evident in certain cell types. To do this, we assign a cell type label to each neighbourhood by finding the most abundant cell type within cells in each neighbourhood (after all, neighbourhoods are in most cases small subpopulations within the same cell type). We can label neighbourhoods in the results data.frame using the function milo.annotate_nhoods. This also saves the fraction of cells harbouring the label.


In [ ]:
milo.annotate_nhoods(mdata, anno_col="T_subtype")

We can see that for the majority of neighbourhoods, almost all cells have the same cell type label. We can rename neighbourhoods where less than 60% of the cells have the top label as “Mixed”

In [ ]:
plt.hist(mdata["milo"].var["nhood_annotation_frac"], bins=30)
plt.xlabel("celltype fraction")

In [ ]:
# Assign "Mixed" where nhood_annotation_frac < 0.6
# mdata["milo"].var.loc[mdata["milo"].var["nhood_annotation_frac"] < 0.6, "nhood_annotation"] = "Mixed"

In [ ]:
fig, ax = plt.subplots(figsize=(7, 4.5))
milo.plot_da_beeswarm(mdata, alpha=0.1,show=False)
plt.tight_layout()
plt.ylabel('')
plt.savefig("Results/11.TCell/11.T_Milo_Abundance.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()

This shows that neighbourhoods of Plasmablast cells, malignant B cells and monocytes are especially enriched in cells from COVID-19 samples.

We can check the effect size by visualizing the cell counts directly

In [ ]:
## Get IDs of plasmablast neighbourhood
pl_nhoods = mdata["milo"].var_names[
    (mdata["milo"].var["SpatialFDR"] < 0.5) & (mdata["milo"].var["nhood_annotation"] == "CD8⁺ GZMK⁺ Tpex")
]

## Visualize cell counts by condition (x-axis) and individuals on all neighbourhoods
milo.plot_nhood_counts_by_cond(mdata, test_var="EBV_status", subset_nhoods=pl_nhoods, log_counts=False)

In [ ]:
## Get IDs of plasmablast neighbourhood
pl_nhoods = mdata["milo"].var_names[
    (mdata["milo"].var["SpatialFDR"] < 0.5) & (mdata["milo"].var["nhood_annotation"] == "CD8⁺ GZMB⁺ Tex")
]

## Visualize cell counts by condition (x-axis) and individuals on all neighbourhoods
milo.plot_nhood_counts_by_cond(mdata, test_var="EBV_status", subset_nhoods=pl_nhoods, log_counts=False)


**<span style="font-size:16px;">Session information：</span>**

In [ ]:
import sys
import platform
import pkg_resources

# Get Python version information
python_version = sys.version
# Get operating system information
os_info = platform.platform()
# Get system architecture information
architecture = platform.architecture()[0]
# Get CPU information
cpu_info = platform.processor()
# Print Session information
print("Python version:", python_version)
print("Operating system:", os_info)
print("System architecture:", architecture)
print("CPU info:", cpu_info)

# Print imported packages and their versions
print("\nImported packages and their versions:")
for package in pkg_resources.working_set:
    print(package.key, package.version)